In [2]:
#importing the required files from the file folder
import pandas as pd
s=pd.read_json("/content/drive/My Drive/NLP/Cell_Phones_and_Accessories.json")
es=pd.DataFrame(s)

#getting the overview of the dataset
es.head()

Rating  Rev_verify  Review_Date  ... Review_timestamp Useful Prod_img
0       5        True   09 1, 2015  ...       1441065600   None     None
1       5        True   01 9, 2016  ...       1452297600   None     None
2       5        True  04 21, 2013  ...       1366502400   None     None
3       3        True  02 27, 2013  ...       1361923200      3     None
4       4        True  12 19, 2013  ...       1387411200   None     None

[5 rows x 11 columns]

In [3]:
#lets see the datatype
es.dtypes

Rating               int64
Rev_verify            bool
Review_Date         object
IC                  object
Prod_meta           object
Reviewer_Name       object
Review              object
Rev_summ            object
Review_timestamp     int64
Useful              object
Prod_img            object
dtype: object

In [4]:
#dropping column unnecessary for the process
es.drop(['Reviewer_Name','Review_timestamp','Prod_img','Review_Date'],axis=1,inplace=True)

#getting the null values in dataset
es.isnull().sum()

Rating             0
Rev_verify         0
IC                 0
Prod_meta     352624
Review           530
Rev_summ         355
Useful        698250
dtype: int64

In [5]:
#Prod_meta and Useful are the features having huge number of null data,so its better to drop these rather than imputing
es.drop(['Prod_meta','Useful'],axis=1,inplace=True)

#Dropping the rest null values from the data
es.dropna(inplace=True)

In [6]:
es.head()

Rating  ...                               Rev_summ
0       5  ...                   He's very happy with
1       5  ...                             Five Stars
2       5  ...  As long as you know how to put it on!
3       3  ...                      Good case overall
4       4  ...                        Awesome Battery

[5 rows x 5 columns]

In [7]:
#checking the count of diffrent values in column rev_verify 
es['Rev_verify'].value_counts()

True     664660
False     94942
Name: Rev_verify, dtype: int64

In [8]:
#its better to process only true reviews coz false reviews has no genuinity
es=es[(es['Rev_verify']==True)]
es.head()

Rating  ...                               Rev_summ
0       5  ...                   He's very happy with
1       5  ...                             Five Stars
2       5  ...  As long as you know how to put it on!
3       3  ...                      Good case overall
4       4  ...                        Awesome Battery

[5 rows x 5 columns]

In [9]:
#as we filter out true reviews,so there is no need of rev_verify column
es.drop(["Rev_verify"],axis=1,inplace=True)

### Data Cleaning and Preprocessing

In [10]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

#combining both text column for ease
es['text'] = es['Review'] + es['Rev_summ']
es['text'] = es['text'].apply(lambda x:pre_process(x))

#show the second 'text' just for fun
es['text'][2]

'saved me lots of money it s not gorilla glass so be careful as it will be subject to easier scratching it also doesn t have sticky stuff on the glass like the original if you press hard on the glass the digitizer will go crazy just shut the screen on and off and you ll be fine as long as you know how to put it on '

In [11]:
#importing stopwords and set an instance for it
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
#importing count vectorizer and applying to the data converted to list
from sklearn.feature_extraction.text import CountVectorizer

#conversion of data to list
docs=es['text'].tolist()
cv=CountVectorizer(max_df=0.85,stop_words=stop,max_features=10000)
word=cv.fit_transform(docs)

#getting shape after applying
word.shape

(664660, 10000)

In [13]:
#lets check the key from the text
list(cv.vocabulary_.keys())[:10]

['bought',
 'husband',
 'happy',
 'great',
 'screen',
 'protector',
 'even',
 'seem',
 'though',
 'therefive']

In [14]:
#getting feature names in limited range to check
list(cv.get_feature_names())[1000:1015]

['brilliantly',
 'bring',
 'bringing',
 'brings',
 'british',
 'brittle',
 'bro',
 'broad',
 'broadcast',
 'broke',
 'broken',
 'brokenone',
 'brokeone',
 'bronze',
 'brother']

In [15]:
#implying tf-idf
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [16]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #creating a tuples of feature,score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [17]:
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs[10]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)


print("\n=====Text=====")
print(docs[0:10])
print("\n===Keywords===")
for k in keywords:
    print(k)


=====Text=====
['bought it for my husband he s very happy with ithe s very happy with', 'great screen protector doesn t even seem as though it s on therefive stars', 'saved me lots of money it s not gorilla glass so be careful as it will be subject to easier scratching it also doesn t have sticky stuff on the glass like the original if you press hard on the glass the digitizer will go crazy just shut the screen on and off and you ll be fine as long as you know how to put it on ', 'the material and fit is very nice but the color is more of a neon green than i expected or would have liked good case overall', 'this last me about days till i have to charge it it does take forever to charge so make sure you plug it in early at night so it will be fully charged in the morning sometimes i will get home late am and when i wake up am hours it will still be charging only at and it will take another to be fully charged but if i have to go somewhere after i wake up it won t be fully charged anywa

In [18]:
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    
    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['Summary','keywords'])
df

Summary                                           keywords
0       bought it for my husband he s very happy with ...  {'happy': 0.793, 'husband': 0.512, 'bought': 0...
1       great screen protector doesn t even seem as th...  {'therefive': 0.722, 'seem': 0.376, 'though': ...
2       saved me lots of money it s not gorilla glass ...  {'glass': 0.43, 'subject': 0.29, 'digitizer': ...
3       the material and fit is very nice but the colo...  {'neon': 0.514, 'green': 0.385, 'liked': 0.325...
4       this last me about days till i have to charge ...  {'fully': 0.504, 'charged': 0.356, 'wake': 0.3...
...                                                   ...                                                ...
664655  i love this nokia lumia unlocked phone fast an...  {'nokia': 0.602, 'lumia': 0.313, 'windows': 0....
664656                            great productfive stars  {'productfive': 0.884, 'great': 0.343, 'stars'...
664657  this iphone case is very durable and long last...  {'producti': 0.621, 'lasting': 0.461, 'love': ...
664658                                    greatfive stars               {'greatfive': 0.933, 'stars': 0.359}
664659  so so it did not look like the picture adverti...  {'cam': 0.568, 'hole': 0.419, 'picture': 0.366...

[664660 rows x 2 columns]

In [19]:
df['Itemcode']=es['IC']
df['Max_Rating']=es['Rating'].max()
df['Avg_Rating']=es['Rating'].unique().mean()
df['Min_Rating']=es['Rating'].min()

In [20]:
df = df[['Itemcode','Summary','keywords','Max_Rating','Min_Rating','Avg_Rating']]
df.head()

Itemcode  ... Avg_Rating
0  B009XD5TPQ  ...        3.0
1  B016MF3P3K  ...        3.0
2  B008DC8N5G  ...        3.0
3  B0089CH3TM  ...        3.0
4  B00AKZWGAC  ...        3.0

[5 rows x 6 columns]

In [21]:
df['Itemcode'].drop_duplicates(inplace=True)
df.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
df.head()

Itemcode  ... Avg_Rating
0  B009XD5TPQ  ...        3.0
1  B016MF3P3K  ...        3.0
2  B008DC8N5G  ...        3.0
3  B0089CH3TM  ...        3.0
4  B00AKZWGAC  ...        3.0

[5 rows x 6 columns]

In [ ]:
df.to_json(r'/content/drive/My Drive/NLP/Accesories.json')

In [25]:
#saving the model using joblib
from sklearn.externals import joblib
joblib.dump(tfidf_transformer,'Mobile_accesory.pkl')

['Mobile_accesory.pkl']